In [129]:
import pandas as pd

url='https://raw.githubusercontent.com/imdevskp/covid_19_jhu_data_web_scrap_and_cleaning/master/covid_19_clean_complete.csv'

covid=pd.read_csv(url,parse_dates=['Date'])
covid[['Province/State']]=covid[['Province/State']].fillna('')
covid['Active']=covid['Confirmed']-covid['Deaths']-covid['Recovered']
covid[['Confirmed','Deaths','Recovered','Active']]=covid[['Confirmed','Deaths','Recovered','Active']].fillna(0)


## Latest Data

In [130]:
temp=covid.groupby(['Country/Region', 'Province/State'])['Confirmed', 'Deaths', 'Recovered', 'Active','Date'].max().sort_values('Confirmed',ascending=False)
temp.style.background_gradient(cmap='Blues')

,,Confirmed,Deaths,Recovered,Active,Date
Country/Region,Province/State,,,,,
China,Hubei,67800,3139,58946,50633,2020-03-21 00:00:00
Italy,,53578,4825,6072,42681,2020-03-21 00:00:00
Spain,,25374,1375,2125,21874,2020-03-21 00:00:00
Germany,,22213,84,233,21896,2020-03-21 00:00:00
Iran,,20610,1556,7635,11466,2020-03-21 00:00:00
France,France,14282,562,12,13708,2020-03-21 00:00:00
US,New York,11710,60,0,11650,2020-03-21 00:00:00
South Korea,,8799,102,1540,7577,2020-03-21 00:00:00
Switzerland,,6575,75,15,6485,2020-03-21 00:00:00


In [131]:
covid_country=covid.groupby(['Date','Country/Region']).sum().loc[covid.Date.max()].sort_values('Confirmed',ascending=False)[['Confirmed','Deaths','Recovered','Active']].reset_index()

## Corona Virus Broken Down by Country

In [132]:
covid_country.style.background_gradient(cmap='Blues')

,Country/Region,Confirmed,Deaths,Recovered,Active
0,China,81305,3259,71857,6189
1,Italy,53578,4825,6072,42681
2,US,25489,307,0,25182
3,Spain,25374,1375,2125,21874
4,Germany,22213,84,233,21896
5,Iran,20610,1556,7635,11419
6,France,14431,562,12,13857
7,South Korea,8799,102,1540,7157
8,Switzerland,6575,75,15,6485
9,United Kingdom,5067,234,67,4766


## Corona Virus Deaths Broken Down by Country 

In [133]:
covid_country[['Country/Region','Deaths']].sort_values('Deaths',ascending=False).reset_index(drop=True).style.background_gradient(cmap='Reds')

,Country/Region,Deaths
0,Italy,4825
1,China,3259
2,Iran,1556
3,Spain,1375
4,France,562
5,US,307
6,United Kingdom,234
7,Netherlands,137
8,South Korea,102
9,Germany,84


## Latest Chinese Data by Province

In [134]:
temp.loc['China'].reset_index()[['Province/State','Confirmed','Deaths','Recovered','Active']].sort_values('Confirmed',ascending=False).style.background_gradient(cmap='Blues')

,Province/State,Confirmed,Deaths,Recovered,Active
0,Hubei,67800,3139,58946,50633
1,Guangdong,1400,8,1325,991
2,Henan,1273,22,1250,881
3,Zhejiang,1236,1,1219,883
4,Hunan,1018,4,1014,670
5,Anhui,990,6,984,758
6,Jiangxi,935,1,934,712
7,Shandong,764,7,749,464
8,Jiangsu,631,0,631,436
9,Chongqing,576,6,570,423


## Map of Confirmed Cases Across the World

In [135]:
import folium

In [136]:
full_latest=covid[covid.Date==max(covid.Date)]

In [137]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=4, zoom_start=2)

for i in range(0, len(full_latest)):
    folium.Circle(
        location=[full_latest.iloc[i]['Lat'], full_latest.iloc[i]['Long']],
        color='crimson', 
        tooltip =   '<li><bold>Country : '+str(full_latest.iloc[i]['Country/Region'])+
                    '<li><bold>Province : '+str(full_latest.iloc[i]['Province/State'])+
                    '<li><bold>Confirmed : '+str(full_latest.iloc[i]['Confirmed'])+
                    '<li><bold>Deaths : '+str(full_latest.iloc[i]['Deaths'])+
                    '<li><bold>Recovered : '+str(full_latest.iloc[i]['Recovered']),
        radius=int(full_latest.iloc[i]['Confirmed'])**1.1).add_to(m)
m

In [138]:
import plotly.express as px
import numpy as np

## Map of Confirmed Cases Across the World by Country

In [139]:
fig = px.choropleth(covid_country, locations="Country/Region", 
                    locationmode='country names', color=np.log(covid_country["Confirmed"]), 
                    hover_name="Country/Region", hover_data=['Confirmed'],
                    color_continuous_scale="Blues", 
                    title='Countries with Confirmed Cases')
fig.update(layout_coloraxis_showscale=True)
fig.show()

In [140]:
formated_gdf=covid.groupby(['Date','Country/Region'])['Confirmed','Deaths','Recovered','Active'].max()

formated_gdf = formated_gdf.reset_index()
formated_gdf['Date'] = pd.to_datetime(formated_gdf['Date'])
formated_gdf['Date'] = formated_gdf['Date'].dt.strftime('%m/%d/%Y')
formated_gdf['size'] = formated_gdf['Confirmed'].pow(0.3)

fig = px.scatter_geo(formated_gdf, locations="Country/Region", locationmode='country names', 
                     color="Confirmed", size='size', hover_name="Country/Region", 
                     range_color= [0, max(formated_gdf['Confirmed'])+2], 
                     projection="natural earth", animation_frame="Date", 
                     title='Spread over time')
fig.update(layout_coloraxis_showscale=False)
fig.show()